In [1]:
from sympy import symbols, simplify,together, I, Poly,expand
# 電気回路のインピーダンス計算が大変なので作ったprogram
# 20250906 shimojo makoto (下条誠)

# 変数をシンボルとして定義
# s: ラプラス演算子 (Laplace operator)
RA,LA,RO,RL, LS, CL,CJ,K, s = symbols('RA,LA,RO,RL, LS, CL,CJ,K, s', positive=True)
# 回路素子のインピーダンスをsで定義
Z_RA = RA           # 抵抗のインピーダンス
Z_LA = s * LA       # コイルのインピーダンス
Z_RO = RO           # 抵抗のインピーダンス
Z_RL = RL           # 抵抗のインピーダンス
Z_LS = s * LS       # コイルのインピーダンス
Z_CL = 1 / (s * CL)     # コンデンサのインピーダンス
Z_CJ = 1 / (s * CJ)     # コンデンサのインピーダンス

def series_impedance_fraction(*impedances): #直列接続を計算し、
    # 分数式で出力する関数。合計を計算し、together()で分数式にまとめる
    return together(sum(impedances))

def parallel_impedance_fraction(*impedances): #並列接続を計算し、
    # 分数式で出力する関数。逆数の和を計算
    reciprocal_sum = sum(1/z for z in impedances)
    # その逆数をとり、together()で分数式にまとめる
    return together(1 / reciprocal_sum)

# RLCの数式を計算( Laplace transform)
GA=1/(series_impedance_fraction(Z_RA,Z_LA))
GO=(parallel_impedance_fraction(Z_RO,Z_CJ))

#Loop Gain (展開してまとめる)
G=simplify(together(expand(K*GA*GO)/(1+K*K*GA*GO)))

# 分子と分母を多項式として整理(ただの印刷用です。なくても可)
numerator, denominator = G.as_numer_denom()
#G_numera=Poly(numerator,s) #多項式object
#G_denomi=Poly(denominator,s)#多項式object
G_numera=Poly(numerator,s).as_expr() #普通の式となる
G_denomi=Poly(denominator,s).as_expr()#普通の式となる
print(f"LoopGain分子: {G_numera}")
print(f"LoopGain分母: {G_denomi}")
GG=G_numera/G_denomi #今は印刷用に使うだけ
print("-" * 20)#--------------------

# ----------------------------------------------------
# ここから具体的な数値を代入する部分
# ----------------------------------------------------
# 代入する値の辞書 (sを含まない)
values = {
    RA:0.299,LA:0.08241e-3,RO:1.0e7,RL: 1.0e5,LS: 0.05,
    CL: 1.0e-3,CJ: 1.42e-5,K: 0.0302
}

# .subs()メソッドを使って数式に値を代入
# この時、sはシンボルとして残ります
G_loopGain_with_values = G.subs(values)

# 数値代入後の数式を表示
print("--- RLC並列回路の合成インピーダンス（sを含む式） ---")
print(f"代入値: RA={values[RA]}, LA={values[LA]}, CJ={values[CJ]},RO={values[RO]}")
print(f"計算結果G: {G_loopGain_with_values}")
print("-" * 20)

LoopGain分子: K*RO
LoopGain分母: CJ*LA*RO*s**2 + K**2*RO + RA + s*(CJ*RA*RO + LA)
--------------------
--- RLC並列回路の合成インピーダンス（sを含む式） ---
代入値: RA=0.299, LA=8.241e-05, CJ=1.42e-05,RO=10000000.0
計算結果G: 302000.0/(0.01170222*s**2 + 42.45808241*s + 9120.699)
--------------------


In [2]:
GA

1/(LA*s + RA)